In [72]:
#Do not run, except you want to reset the list

article_urls = []

In [2]:
#IMPORT DEPENDENCIES

import newspaper
from newspaper import Article
from newspaper import fulltext
import nltk

import requests
import re

import csv

import pandas as pd
import numpy as np

from time import time
from urllib.request import urlopen

from pyspark import SparkContext
from pyspark.sql import SparkSession

In [3]:
try:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)
except:
    pass

In [73]:
shell_search_webpages = [
                   'https://www.theguardian.com/',
                   'https://www.newsnow.co.uk/',


                  ]



In [74]:
#Extract individual urls in the wepage and append them to the article_urls list
#if "active_url" block was reset--in the first code block,un-comment the "memoize_articles" to prevent caching

def fetch_links(webpage):
    for webpage in webpage:
        news_portal = newspaper.build(webpage,memoize_articles=False)
        for article in news_portal.articles:
            article_urls.append(article.url) 
            

In [75]:
fetch_links(shell_search_webpages)

In [76]:
len(article_urls)

180

In [27]:
print(article_urls)

['https://www.theguardian.com/mobile/2014/may/29/the-guardian-for-mobile-and-tablet', 'https://www.theguardian.com/world/live/2022/sep/16/russia-ukraine-war-zelenskiy-compares-scenes-in-izium-to-bucha-massacre-live-updates', 'https://www.theguardian.com/world/2022/sep/16/ukraine-mass-grave-with-440-bodies-discovered-in-recaptured-izium-says-police-chief', 'https://www.theguardian.com/us-news/2022/sep/15/judge-raymond-drearie-trump-special-master-records-case', 'https://www.theguardian.com/uk-news/live/2022/sep/16/queen-elizabeth-ii-lying-in-state-westminster-hall-king-charles-iii-mourners-queue-london-live-news-latest-updates', 'https://www.theguardian.com/uk-news/2022/sep/16/welsh-first-minister-says-people-have-right-to-protest-during-king-charles-visit', 'https://www.theguardian.com/uk-news/2022/sep/16/chinese-delegation-banned-from-queens-lying-in-state', 'https://www.theguardian.com/uk-news/2022/sep/16/eight-of-queens-grandchildren-to-stand-vigil-prince-william-harry', 'https://ww

In [77]:
#Filter articles based on the 'shell' keyword
article_urls_filter = list(filter(lambda k: 'shell' in k, article_urls))

len(article_urls_filter)

0

In [78]:
#write/append available articles to csv (articles_urls_filter)

# open the file in the write mode
with open('./webpages_links.csv', 'w') as f:
    # create the csv writer
    writer = csv.writer(f,delimiter = '\n',skipinitialspace=True,lineterminator='\n')

    # write a row to the csv file
    writer.writerow(article_urls)

In [83]:
#FETCHING ARTICLES' PROPERTIES AND BASIC CLEANING

start_time = time()

url_list_path = './webpages_links.csv'

urls_lines = sc.textFile(url_list_path)

output_data = []

def processRecord(url):
    global output_data
    if len(url) > 0:
        try:
            article = Article(str(url)) #pass the url through the Article object
            article.download() #download the published article of the url

            #NLP preparation
            article.parse() #prepare the article for nlp extractions
            article.download('punkt') #download 'punkt' to enable tokenization
            article.nlp() #enable Natural Language Processing

            #Extract html text
            html_text = requests.get(url).text
            text = fulltext(html_text)

            #Remove HTML tags
            TAG_RE = re.compile(r'<[^>]+>')
            clean_text = TAG_RE.sub('', text)
            new_text = ' '.join(clean_text.splitlines())

            #Remove square brackets
            authors = ','.join(map(str,article.authors))
            keywords = ','.join(map(str,article.keywords))


            output = [article.title,authors,url,new_text,keywords,article.publish_date]
            

        except requests.exceptions.Timeout:
            pass  # handle the timeout    
        except requests.exceptions.RequestException:
            pass  # handle the exception. maybe wait and try again later
        except AttributeError:
            pass
        except ArticleException:
            pass

    return output



temp = urls_lines.map(processRecord)

temp_rdd = temp.collect()

for elem in temp_rdd:
    output_data.append(elem)
    #print(elem)

    
    
print("Time taken: " + str(time() - start_time))


Time taken: 212.7148950099945


**SEND NOTIFICATION FOR SUCCESS OR FAILURE OF URLS EXTRACTION (IF POSSIBLE, WITH THE NUMBERS OF URLS EXTRACTED)

In [84]:
len(output_data)

180

**SEND NOTIFICATION FOR SUCCESS OR FAILURE OF ARTICLE PROPERTIES EXTRACTION (IF POSSIBLE, WITH THE NUMBERS OF SUCCESSFUL ONES & REASONS FOR FAILURE)

In [86]:
df

,0
0,None
1,None
2,None
3,None
4,None
...,...
175,None
176,None
177,None
178,None


In [87]:
df = pd.DataFrame(output_data)
df.columns = ["headlines","authors","url","text","keywords","publish_date"]

In [88]:
df

,headlines,authors,url,text,keywords,publish_date
0,Introducing the new Guardian app for Android a...,,https://www.theguardian.com/mobile/2014/may/29...,As it happens not after it happened Get the l...,"ios,premium,faster,android,unfold,latest,guard...",2014-05-29
1,Russia-Ukraine war live: reports of mass buria...,"Vivian Ho,Martin Belam,Michael Coulter",https://www.theguardian.com/world/live/2022/se...,From 5h ago 07.23 Reports of mass burial site ...,"ukrainian,office,killed,russian,kherson,russia...",2022-09-16
2,Ukraine mass burial site with 440 bodies disco...,Luke Harding,https://www.theguardian.com/world/2022/sep/16/...,Volodymyr Zelenskiy has accused Russia of “lea...,"mass,burial,440,discovered,graves,bodies,city,...",2022-09-16
3,Judge proposed by Trump named special master i...,Maanvi Singh,https://www.theguardian.com/us-news/2022/sep/1...,"A federal judge has named Raymond Dearie, a se...","federal,seized,records,trumps,trump,case,maral...",2022-09-15
4,Queen lying in state: King visits Wales as ent...,"Léonie Chao-Fong,Rebecca Ratcliffe",https://www.theguardian.com/uk-news/live/2022/...,From 47m ago 11.12 King arrives in Wales The K...,"photograph,service,king,visits,lying,live,queu...",2022-09-16
...,...,...,...,...,...,...
175,The Guardian :: What can I expect after attend...,Thirtythree Llp,https://workforus.theguardian.com/index.php/fa...,Already applied for a job at Guardian News & M...,"job,expect,attending,guardian,interview,media,...",NaT
176,The Guardian :: How do I make an accessibility...,Thirtythree Llp,https://workforus.theguardian.com/index.php/fa...,Already applied for a job at Guardian News & M...,"job,request,guardian,media,applied,accessibility",NaT
177,Flats for rent in Yorkshire and the Humber,,https://www.newsnow.co.uk/classifieds/houses-f...,By clicking “OK” or continuing to use this sit...,"set,policyok,rent,read,yorkshire,flats,privacy...",NaT
178,Flats for rent in Yorkshire and the Humber,,http://www.newsnow.co.uk/classifieds/houses-fl...,By clicking “OK” or continuing to use this sit...,"set,policyok,rent,read,yorkshire,flats,privacy...",NaT


In [67]:
guardian_publications = df.to_csv('./guardian_publications.csv')